In [84]:
# Install a conda package in the current Jupyter kernel
import sys  #https://www.quora.com/What-does-import-sys-mean-in-Python
#sys es uno de los modulos disponibles para importar
# sys module: conjunto de funciones que brinda la info de como el python script interactua con el host system

# sys modules has information about :
#    1. version of Python is running
#    2. The path to the Python executable that is executing your script
#    3. The command line options used to execute your script
#    4. Any Python specific flags that have been set
#    5. Information about floating point values and their representation.
#    6. functions which can set what happens after each python instruction.
#    7. function to cleanly exit the Python interpreter.

#If your code has import sys - then it can use the functionality and data in it by using terms such as :

# sys.argv: A list of strings which is the command line used to execute the script separated by spaces.
# sys.exit(): Exit Python cleanly
# sys.version: The Python version string

!conda install --yes --prefix {sys.prefix} -c conda-forge poppler
!pip install pdf2image

import boto3 # --> permite usar bucket S3!!!
#Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python,
# que permite a los Python DEV escribir software que use los servicios de: Amazon S3 and Amazon EC2.
import io # --> a web-based platform for extracting data from websites without writing any code
import json
#El formato JSON se utiliza para estructurar datos en forma de texto y permite el intercambio de información 
#entre aplicaciones de manera sencilla, liviana y rápida
#Los datos en formato JSON pueden ser utilizados por prácticamente todos los lenguajes de programación 
#(como Java, C#, C, C++, PHP, JavaScript, Python, etc.)


import pandas as pd
import re #--> Operaciones con EXPRSIONES REGULARES
import requests #--> La librería para hacer peticiones http en Python
import logging #--> logging de Python registra errores de código simples y genera un mensaje
import time
from pdf2image import convert_from_path

#from PIL import Image

pd.set_option('display.max_columns', None)

Solving environment: done

# All requested packages already installed.



In [85]:
import boto3
import io
import json
import numpy as np
import logging
import os
import pandas as pd
import pdf2image
import requests
from concurrent import futures

In [86]:
ENDPOINT_NAME = 'n2-data-certification-model-deploy-dev-endpoint'  # URL's que reciben o retornan información de un Web API se les llama endpoints 
MODEL_VERSION = '13'
MAX_LABELS = 200 #LABEL: etiqueta para un elemento en una interfaz de usuario
MIN_CONFIDENCE = 5 #al menos 5% --> esto responde a la cantidad MINIMA de "confianza" en la que el label esta en la imagen
FEATURES = ['Text', 'Handwriting', 'Symbol', 'Number', 'Paper', 'Document', 'Apparel', 'Clothing', 'Signature', 'Autograph', 
            'Animal', 'Letter', 'Alphabet', 'Page', 'Plant', 'Accessories', 'Accessory', 'Food', 'Envelope', 'Beverage', 
            'Drink', 'Electronics', 'Musical Instrument', 'Word', 'File Binder', 'Alcohol', 'Mail', 'Footwear', 'Invertebrate', 'Furniture', 
            'QR Code', 'Gray', 'Calligraphy', 'Jewelry', 'Broom']
#feature is a measurable property of the object you're trying to analyze. 
#In datasets, features appear as columns

client_rekognition = boto3.client("rekognition", 'us-east-1') #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#client
#class rekognition.client --> viene con los objetos (funciones: DetectFaces,DetectLabels,DetectText)
client_sagemaker = boto3.client("runtime.sagemaker")
client_s3 = boto3.resource('s3')

In [87]:
import base64 
import re

In [148]:
#pdf prueba en base 64
url_s3='https://stage-upload-management-service.s3.amazonaws.com/384ca7bc-728f-4bfb-9cbf-99e70c69d8ea-AUTOREGISTRO.docx.pdf'

response = requests.get(url_s3)

In [129]:
#print(type(in_64.decode("utf-8")))

In [150]:
in_64=response.content.decode("utf-8")
result = re.sub(r"data.+base64.\b",  ' ',  in_64)

    
x = re.findall((r"(pdf;|png;|jpeg;|bmp;)\b") , in_64) #findall devuelve lista con los elementos encontrados
#uso el ; al final de pdf,png,etc pues necesito garantizar que en el resto del .txt NO exista otras 3 letras que en realidad no representen un tipo de formato de archivo
print(x)

if ("pdf;" in x):
    with open('pdf_file.pdf', 'wb') as theFile:
        theFile.write(base64.b64decode(result))
        print(type(base64.b64decode(result)))
elif("png;" in x):
    with open('file_png.png', 'wb') as theFile:
        theFile.write(base64.b64decode(result))
        print(type(base64.b64decode(result)))
elif("jpeg;" in x):
    with open('file_jpg.jpg', 'wb') as theFile:
        theFile.write(base64.b64decode(result))
        print(type(base64.b64decode(result)))
elif("bmp;" in x):
    with open('file_bmp.bmp', 'wb') as theFile:
        theFile.write(base64.b64decode(result))
        print(type(base64.b64decode(result)))



['pdf;']
<class 'bytes'>


In [141]:
#Puesto que el usuario puede cargar libremente el formato del certificado
# detect_labels_from pdf --> aplica cuando carga PDF
# detect_labels_from bytes --> - - - OTRO FORMATO
#
# funcion detect_labels --> engloba a los 2

# A tener en cuenta: el usuario puede cargar HASTA 3 archivos distintos como certificados
# Sin embargo, este modelo SOLO evalua la primer pagina de los mismos

def detect_labels_from_pdf(client, pdf_file, max_labels, min_confidence):
    response = requests.get(pdf_file, timeout=30)  #.get() method sends a GET request to the specified url
    #timeout= seconds to wait for the client to make a connection/send a response
    pages = pdf2image.convert_from_bytes(response.content, dpi=300)
    #convert_from_bytes(pdf.read()) --> convierte pdf en imagen
    #pdf.read() --> en este caso se RESPONSE.CONTENT
    img = pages[0].convert('RGB') # PAGES[0] 
    buf = io.BytesIO() #instancia vacía que dsps la cargo con el contenido de la siguiente linea de codigo 
    img.save(buf, format='JPEG')
    byte_im = buf.getvalue()
    return client.detect_labels(Image={'Bytes': byte_im}, MaxLabels=max_labels, MinConfidence=min_confidence)
    #devuelve un DICCIONARIO con los labels y sus parametros:
    # {'Labels':
    #    [{'Name': 'QR Code', 
    #      'Confidence': 91.76808166503906, 'Instances': [], 'Parents': []}, etc 
    
def detect_labels_from_bytes(client, file, max_labels, min_confidence):
    #cuando el documento no es pdf
    response = requests.get(file)
    
    return client.detect_labels(Image={'Bytes': response.content}, MaxLabels=max_labels, MinConfidence=min_confidence)

#Este caso engloba los 2 casos anteriores!: cuando el usuario carga el certif como PDF o como OTRO FORMATO
def detect_labels(client, file, max_labels, min_confidence):
    try:
        file_extension = file.split('.')[-1].lower() #.split()[-1] pues ahi queda el FORMATO (.pdf/.JPEG/..)
        if file_extension == 'pdf':
            return detect_labels_from_pdf(client, file, max_labels, min_confidence)
        else:
            return detect_labels_from_bytes(client, file, max_labels, min_confidence)
    except: #caso que no pueda levantar el archivo por formato incompatible --> que tire un error
        logging.error("Exception occurred", exc_info=True)
        return {}
    
def query_endpoint(encoded_tabular_data, endpoint_name, content_type='text/csv'):
    response = client_sagemaker.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_tabular_data
    )
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    predicted_probabilities = model_predictions["probabilities"]
    return np.array(predicted_probabilities)

def save_rekognition_labels(id_image, response):
    if (os.getenv('BUCKET_REKOGNITION_LABELS', default = None) is not None) & (os.getenv('FOLDER_REKOGNITION_LABELS', default = None) is not None):
        try:
            s3object = client_s3.Object(os.getenv('BUCKET_REKOGNITION_LABELS'), 
                                                '{}/image_{}.json'.format(os.getenv('FOLDER_REKOGNITION_LABELS'), id_image))
            s3object.put(Body=(bytes(json.dumps(response).encode('UTF-8'))))
        except:
            logging.error("Exception occurred", exc_info=True)

def get_predictions(data_image):
    labels = ["Rechazado", "Aprobado"]
    try:
        logging.info(f'get_predictions id_image: {data_image["id_image"]}')
        # Call rekognition
        logging.info(f'call rekognition id_image: {data_image["id_image"]}')
        response = detect_labels(client_rekognition, data_image['url_image'], max_labels=MAX_LABELS, min_confidence=MIN_CONFIDENCE)
        # Save rekognition features
        logging.info(f'save rekognition features id_image: {data_image["id_image"]}')
        save_rekognition_labels(id_image=data_image['id_image'], response=response)
        # Create dataframe
        logging.info(f'create dataframe id_image: {data_image["id_image"]}')
        df = pd.DataFrame([{i['Name']:i['Confidence'] for i in response['Labels']}])
        columns = df.columns.tolist()
        empty_columns = list(set(FEATURES) - set(columns))
        for column in empty_columns:
            df[column] = 0
        df['file_is_pdf'] = int(data_image['url_image'].split('.')[-1].lower() == 'pdf')
        # Endpoint request
        logging.info(f'call sagemaker id_image: {data_image["id_image"]}')
        query_response_batch = query_endpoint(
            df[['file_is_pdf'] + FEATURES].to_csv(header=False, index=False).encode("utf-8"),
            endpoint_name=ENDPOINT_NAME
        )
        predict_prob = np.concatenate(parse_response(query_response_batch), axis=0)
        logging.info(f'response id_image: {data_image["id_image"]}')
        return {
            "id_image": data_image["id_image"],
            "url_image": data_image["url_image"],
            "score": predict_prob[1],
            "prediction_label": labels[np.argmax(predict_prob)],
            "model_version_cla": MODEL_VERSION,
            "end_point": ENDPOINT_NAME,
            "prediction_status": "Ok"
        }
    except Exception as err:
        logging.error("Exception occurred", exc_info=True)
        return {
            "id_image": data_image["id_image"],
            "url_image": data_image["url_image"],
            "score": 0,
            "prediction_label": "None",
            "model_version_cla": MODEL_VERSION,
            "end_point": ENDPOINT_NAME,
            "prediction_status": format(err)
        }

In [153]:
#get_predictions(data_image={'id_image':2879,
#                            "url_image": "https://prod-gcba-us-east-1-upload.s3.amazonaws.com/us-east-1%3Aa55bc7d5-0df0-4f0d-a6ea-8657da1ebb1b/us-east-1%3Aa28bffa3-7707-461b-9265-cfacc6829d9a/certificates/1617395950/UMA.PDF"})

get_predictions(data_image={'id_image':2879,
                            "url_image": "/MIA/Modelo de certificados/pdf_file.pdf"})


ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-141-ba128e612e2c>", line 38, in detect_labels
    return detect_labels_from_pdf(client, file, max_labels, min_confidence)
  File "<ipython-input-141-ba128e612e2c>", line 14, in detect_labels_from_pdf
    pages = pdf2image.convert_from_bytes(response.content, dpi=300)
AttributeError: 'str' object has no attribute 'content'
ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-141-ba128e612e2c>", line 77, in get_predictions
    df = pd.DataFrame([{i['Name']:i['Confidence'] for i in response['Labels']}])
KeyError: 'Labels'


{'id_image': 2879,
 'url_image': '/MIA/Modelo de certificados/pdf_file.pdf',
 'score': 0,
 'prediction_label': 'None',
 'model_version_cla': '13',
 'end_point': 'n2-data-certification-model-deploy-dev-endpoint',
 'prediction_status': "'Labels'"}

In [151]:
#data_image={'id_image':2879,
#            "url_image": "https://stage-upload-management-service.s3.amazonaws.com/384ca7bc-728f-4bfb-9cbf-99e70c69d8ea-AUTOREGISTRO.docx.pdf"}


data_image={'id_image':2879,
            "url_image": "MIA/Modelo de certificados/pdf_file.pdf"}

In [152]:
data_image

{'id_image': 2879, 'url_image': 'MIA/Modelo de certificados/pdf_file.pdf'}

In [146]:
response = detect_labels(client_rekognition, data_image['url_image'], max_labels=MAX_LABELS, min_confidence=MIN_CONFIDENCE)

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-141-ba128e612e2c>", line 38, in detect_labels
    return detect_labels_from_pdf(client, file, max_labels, min_confidence)
  File "<ipython-input-141-ba128e612e2c>", line 14, in detect_labels_from_pdf
    pages = pdf2image.convert_from_bytes(response.content, dpi=300)
AttributeError: 'str' object has no attribute 'content'


In [123]:
response

{}

In [124]:
df = pd.DataFrame([{i['Name']:i['Confidence'] for i in response['Labels']}])
df

KeyError: 'Labels'

In [125]:
columns = df.columns.tolist()
empty_columns = list(set(FEATURES) - set(columns))
for column in empty_columns:
    df[column] = 0
df['file_is_pdf'] = int(data_image['url_image'].split('.')[-1].lower() == 'pdf')
df

NameError: name 'df' is not defined

In [126]:
df[['file_is_pdf'] + FEATURES]
#file_is_pdf --> 0 si no es pdf/1 si es pdf
# FEATURES --> tags

NameError: name 'df' is not defined

In [14]:
query_response_batch = query_endpoint(
    df[['file_is_pdf'] + FEATURES].to_csv(header=False, index=False).encode("utf-8"),
    endpoint_name=ENDPOINT_NAME
)
predict_prob = np.concatenate(parse_response(query_response_batch), axis=0)

In [25]:
query_response_batch

In [26]:
predict_prob

In [ ]:
#2 repos: rekognition/conexion a sagemaker
#ramas: dev/stage/qa/prod(=master por default)
#uell-data-rekognition-tags
#uell-data-model-images-classification

#2do paso: levantar instancia de rekognition